In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import re
import plotly as pl 
import matplotlib.pyplot as plt
from pathlib import Path

# Import db related librarys
import psycopg2
import sys
import boto3
import os
import sqlalchemy
import json
import csv
from getpass import getpass

In [2]:
# Configure connection parameters to AWS RDS
def connect_to_db():
    
    connection = None
    #enter_password = getpass('Enter database password')
    
    try:
        print('Connecting to the PostgreSQL database...')
        connection = psycopg2.connect(
            host = "database-chocolate.cafzzay3t2tr.us-east-2.rds.amazonaws.com",
            port = 5432,
            user = 'postgres',
            password = getpass('Enter database password'),
            database = 'postgres'
            )
        print('Connection successful')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return connection



In [3]:
# Call the connect_to_db to connect to database
connection = connect_to_db()


Connecting to the PostgreSQL database...
Enter database password········
Connection successful


In [4]:
# Create cursor to perform database operations
cursor = connection.cursor()

# If curser object = 0, then connection was successfully established
cursor

<cursor object at 0x00000195997AD588; closed: 0>

In [5]:
# Query the location_table from AWS database and store in dataframe - verify there is data in the table.
sql = """
SELECT * FROM location_table
"""

location_table_from_db = pd.read_sql(sql, con=connection)

In [6]:
# Display the location_table queried from database
location_table_from_db

,country_code,latitude,longitude,broad_bean_origin_country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
242,ZM,-13.133897,27.849332,Zambia
243,ZW,-19.015438,29.154857,Zimbabwe
244,HI,19.898682,-155.665857,Hawaii
245,ZZ,0.000000,0.000000,Unknown


In [7]:
# To end transaction
# connection.rollback()

In [8]:
# Query the clean_flavours_table from AWS database and store in dataframe.
sql = """
SELECT * FROM clean_flavors_table
"""

clean_flavors_from_db = pd.read_sql(sql, con=connection)

In [9]:
clean_flavors_from_db

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America
...,...,...,...,...,...,...,...,...,...,...,...,...
1955,Zotter,Peru,647,2011,70.0,Austria,3.75,missing,Peru,"4- B,Sw,C,Sa","creamy, fatty, mild nutty",South America
1956,Zotter,Congo,749,2011,65.0,Austria,3.00,Forastero,Republic of Congo,"4- B,Sw,C,Sa","dairy, salt, caramel",Africa
1957,Zotter,Kerala State,749,2011,65.0,Austria,3.50,Forastero,India,"4- B,Sw,C,Sa","creamy, masculine, earthy",Asia
1958,Zotter,Kerala State,781,2011,62.0,Austria,3.25,missing,India,"4- B,Sw,C,Sa","oily, subdued, caramel, salt",Asia


In [10]:
# Join the 2 tables together on broad_bean_origin_country, select the columns you want to view in df

sql = """ SELECT clean_flavors_table .*, location_table.country_code,location_table.longitude, location_table.latitude
FROM clean_flavors_table
INNER JOIN location_table
ON clean_flavors_table.broad_bean_origin_country = location_table.broad_bean_origin_country;
"""

# Store the joined tables in dataframe
joined_tables = pd.read_sql(sql, con=connection)

# View the new dataframe combined from two sql tables
joined_tables.head(10)

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent,country_code,longitude,latitude
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa,ST,6.613081,0.186360
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa,TG,0.824782,8.619543
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa,TG,0.824782,8.619543
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa,TG,0.824782,8.619543
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America,PE,-75.015152,-9.189967
5,A. Morin,Carenero,1315,2014,70.0,France,2.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
6,A. Morin,Cuba,1315,2014,70.0,France,3.50,missing,Cuba,"4- B,S,C,L","sliglty dry, papaya",Caribbean,CU,-77.781167,21.521757
7,A. Morin,Sur del Lago,1315,2014,70.0,France,3.50,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
8,A. Morin,Puerto Cabello,1319,2014,70.0,France,3.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
9,A. Morin,Pablino,1319,2014,70.0,France,4.00,missing,Peru,"4- B,S,C,L","delicate, hazelnut, brownie",South America,PE,-75.015152,-9.189967


In [11]:
# Close connection to RDS
cursor.close()

In [12]:
# Check that connection is closed where object returned = -1
cursor

<cursor object at 0x00000195997AD588; closed: -1>